Riddler Solution for 1/17/20
Full description of problem can be found at https://fivethirtyeight.com/features/can-you-track-the-delirious-ducks/

# Analytical Solution
We can construct a matrix that describes the probability of reaching each square from any other square on the board. If we lay out the "pond" where each square is assigned a number we have: <br>
|1|2|3| <br>
|4|5|6| <br>
|7|8|9| <br>
Therefore, for the "pond" we have the $9\times9$ matrix: <br>
$$\begin{bmatrix} 0 & 1/3 & 0 & 1/3 & 0 & 0 & 0 & 0 & 0 \\ 1/2 & 0 & 1/2 & 0 & 1/4 & 0 & 0 & 0 & 0 \\ 0 & 1/3 & 0 & 0 & 0 & 1/3 & 0 & 0 & 0 \\ 1/2 & 0 & 0 & 0 & 1/4 & 0 & 1/2 & 0 & 0 \\ 0 & 1/3 & 0 & 1/3 & 0 & 1/3 & 0 & 1/3 & 0 \\ 0 & 0 & 1/2 & 0 & 1/4 & 0 & 0 & 0 & 1/2 \\ 0 & 0 & 0 & 1/3 & 0 & 0 & 0 & 1/3 & 0 \\ 0 & 0 & 0 & 0 & 1/4 & 0 & 1/2 & 0 & 1/2 \\ 0 & 0 & 0 & 0 & 0 & 1/3 & 0 & 1/3 & 0 \end{bmatrix}$$ <br>
Each of the columns of this matrix add to 1. <br>
We then describe the initial state of a duck with the state vector: <br>
$$ s_0 = \begin{bmatrix} 0\\0\\0\\0\\1\\0\\0\\0\\0\\ \end{bmatrix} $$ <br>
Of course, this can be generalized to any starting position of any duck. If we take the dot product of $s_0$ with the probability matrix $T$, we can see that we recover the appropriate probabilties of going to any postion in the pond.

In [1]:
import numpy as np

T = np.array([[0.,1./3.,0.,1./3.,0.,0.,0.,0.,0.],
              [1./2.,0.,1./2.,0.,1./4.,0.,0.,0.,0.],
              [0.,1./3.,0.,0.,0.,1./3.,0.,0.,0.],
              [1./2.,0.,0.,0.,1./4.,0.,1./2.,0.,0.],
              [0.,1./3.,0.,1./3.,0.,1./3.,0.,1./3.,0.],
              [0.,0.,1./2.,0.,1./4.,0.,0.,0.,1./2.],
              [0.,0.,0.,1./3.,0.,0.,0.,1./3.,0.],
              [0.,0.,0.,0.,1./4.,0.,1./2.,0.,1./2.],
              [0.,0.,0.,0.,0.,1./3.,0.,1./3.,0.]])
s0 = np.array([[0],[0],[0],[0],[1],[0],[0],[0],[0]])
print(np.dot(T,s0))

[[0.  ]
 [0.25]
 [0.  ]
 [0.25]
 [0.  ]
 [0.25]
 [0.  ]
 [0.25]
 [0.  ]]


If we take $N$ steps we must raise $T$ to the $N$ power to determine the probability that the duck will be in a given square after that time. As a test, we know that the ducks can take the same first step and meet each other after just 1 step, so if we take the probability that they will meet up after $N$ steps: <br>
$$ P_N = (T^N \cdot s_0)^T \cdot (T^N \cdot s_0) $$ <br>
And take $N=1$, we should get a probability of $1/4$

In [2]:
np.dot(np.transpose((np.dot(np.linalg.matrix_power(T,1),s0))),(np.dot(np.linalg.matrix_power(T,1),s0)))

array([[0.25]])

Which we do. To find the average number of steps it takes for the ducks to converge we can take the weighted average of $N$ where the weight is $P_N$ for each step. However, we must also account for all of the paths where the ducks are likely to have already met up. For example, since $1/4$ of the time they will meet on the first move, there is only a $3/4$ chance that it takes more than 1 move. To account for this, we can also weight each $N$ by the probability it takes at least $N$ steps to get to $N$, which is $1 - \prod_{N=1}^{N-1} P_N$. Therefore our final equation for the average number of steps it takes for the ducks to converge is: <br>
$$ Steps_{avg} = \sum_{N=1}^{\inf} \left( P_N N (1-\prod_{N=1}^{N-1} P_N) \right) $$

In [8]:
avg_steps = 0
N_max = 100
thresh = 1e-10
p_arr = np.zeros(N_max+1)
for i in range(N_max):
    if i == 0:
        continue
        
    p_n = np.dot(np.transpose((np.dot(np.linalg.matrix_power(T,i),s0))),(np.dot(np.linalg.matrix_power(T,i),s0)))[0][0]
    print(p_n)
    diff = (p_n * i) * (1. - np.prod(p_arr,where=p_arr!=0))
    avg_steps += diff
    p_arr[i] = p_n
#     print(diff)
    
    if diff < thresh:
        print(f"Converged under threshold value in {i} steps")
        break
        
print(avg_steps)

0.25
Converged under threshold value in 1 steps
0.0


In [7]:
x = np.zeros(N_max)
x[1] = 0.25
x[2] = 0.19
x[3] = 0.25
x[4] = 0.19
np.prod(x,where=x!=0)

0.00225625

# Simulations